# Maintenance Report NLP Analysis

This notebook implements an NLP pipeline for analyzing maintenance reports using BERT and NLTK.

In [1]:
import pandas as pd
import torch
import nltk
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)

# Set up device (GPU, MPS, or CPU)
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

# Set up NLTK data
nltk.data.find("tokenizers/punkt")
nltk.data.find("corpora/wordnet")

# ... (rest of the code remains the same)

LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - '/Users/nikolay_tishchenko/nltk_data'
    - '/Users/nikolay_tishchenko/codeium/coolsys/coolsys_env/nltk_data'
    - '/Users/nikolay_tishchenko/codeium/coolsys/coolsys_env/share/nltk_data'
    - '/Users/nikolay_tishchenko/codeium/coolsys/coolsys_env/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
# Suppress warnings from the transformers library
logging.set_verbosity_error()

# Download required NLTK data
try:
    nltk.data.find("tokenizers/punkt")
except LookupError:
    nltk.download("punkt", quiet=True)

try:
    nltk.data.find("corpora/wordnet")
except LookupError:
    nltk.download("wordnet", quiet=True)

try:
    nltk.data.find("taggers/averaged_perceptron_tagger")
except LookupError:
    nltk.download("averaged_perceptron_tagger", quiet=True)


In [2]:
import pandas as pd
import torch
import transformers
import nltk
import pyodbc
import logging
import re

from typing import Optional, List, Tuple
import time
import os
from spellchecker import SpellChecker
from functools import lru_cache

import platform

from transformers import BertTokenizer, BertForTokenClassification, pipeline, logging

import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

from contextlib import contextmanager
from typing import Optional, List, Tuple


# Set up logging
import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# Set up device (GPU, MPS, or CPU)
import torch
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    torch.cuda.set_device(0)  # Set the GPU device to use
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

# Set up NLTK data
import nltk
nltk.download("punkt", quiet=True)
nltk.download("wordnet", quiet=True)

True

In [ ]:
import pandas as pd
import torch
import transformers
import nltk
import pyodbc
import logging
import re

from typing import Optional, List, Tuple
import time
import os
from spellchecker import SpellChecker
from functools import lru_cache

import platform

from transformers import BertTokenizer, BertForTokenClassification, pipeline, logging

from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

from contextlib import contextmanager
from typing import Optional, List, Tuple


# Set up logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# Set up device (GPU, MPS, or CPU)
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    torch.cuda.set_device(0)  # Set the GPU device to use
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

# Set up NLTK data
nltk.download("punkt", quiet=True)
nltk.download("wordnet", quiet=True)

In [ ]:
import pandas as pd
import torch
import transformers
import nltk
import pyodbc
import logging
import re

from typing import Optional, List, Tuple
import time
import os
from spellchecker import SpellChecker
from functools import lru_cache

import platform

from transformers import BertTokenizer, BertForTokenClassification, pipeline, logging

import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

from contextlib import contextmanager
from typing import Optional, List, Tuple

In [ ]:
# Set up logging
import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# Set up device (GPU, MPS, or CPU)
import torch
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    torch.cuda.set_device(0)  # Set the GPU device to use
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

# Set up NLTK data
import nltk
nltk.download("punkt", quiet=True)
nltk.download("wordnet", quiet=True)

In [ ]:
# Suppress warnings from the transformers library
logging.set_verbosity_error()

# Download required NLTK data
try:
    nltk.data.find("tokenizers/punkt")
except LookupError:
    nltk.download("punkt", quiet=True)

try:
    nltk.data.find("corpora/wordnet")
except LookupError:
    nltk.download("wordnet", quiet=True)

try:
    nltk.data.find("taggers/averaged_perceptron_tagger")
except LookupError:
    nltk.download("averaged_perceptron_tagger", quiet=True)


In [ ]:
# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Constants
CSV_FILE_PATH = 'data/ice_makers.csv'
DB_CONFIG = {
    'driver': 'ODBC Driver 17 for SQL Server',
    'server': '35.184.99.218',
    'database': 'coolsys',
    'uid': 'sqlserver',
    'pwd': 'Ybz8Vq+|>\H/<2py'
}

SQL_QUERY = """
SELECT
    w.wrkordr_wrk_rqstd,
    w.wrkordr_wrk_prfrmd,
    w2.wrkordreqpmnt_wrk_rqstd,
    w2.wrkordreqpmnt_wrk_prfrmd,
    w3.wrkordrinvntry_dscrptn
FROM
    coolsys.dbo.wrkordr w
    INNER JOIN coolsys.dbo.wrkordrinvntry w3 ON w.wrkordr_rn = w3.wrkordr_rn
    INNER JOIN coolsys.dbo.wrkordreqpmnt w2 ON w.wrkordr_rn = w2.wrkordr_rn
WHERE
    w.wrkordr_wrk_rqstd LIKE '%ICE MACHINE%'
    OR w.wrkordr_wrk_prfrmd LIKE '%ICE MACHINE%'
    OR w2.wrkordreqpmnt_wrk_rqstd LIKE '%ICE MACHINE%'
    OR w2.wrkordreqpmnt_wrk_prfrmd LIKE '%ICE MACHINE%'
    OR w3.wrkordrinvntry_dscrptn LIKE '%ICE MACHINE%';
"""

@contextmanager
def database_connection():
    """Context manager for database connections with proper error handling."""
    conn = None
    try:
        conn_str = (
            f'DRIVER={{{DB_CONFIG["driver"]}}};'
            f'SERVER={DB_CONFIG["server"]};'
            f'DATABASE={DB_CONFIG["database"]};'
            f'UID={DB_CONFIG["uid"]};'
            f'PWD={DB_CONFIG["pwd"]}'
        )
        conn = pyodbc.connect(conn_str, timeout=30)  # Add connection timeout
        logger.info("Database connection established")
        yield conn
    except pyodbc.Error as e:
        logger.error(f"Database connection error: {str(e)}", exc_info=True)
        raise
    finally:
        if conn:
            conn.close()
            logger.info("Database connection closed")

def fetch_data_from_db() -> Tuple[Optional[pd.DataFrame], Optional[str]]:
    """Fetch data from database with optimized performance and error handling."""
    try:
        with database_connection() as conn:
            # Configure connection for better performance
            conn.setdecoding(pyodbc.SQL_CHAR, encoding='utf-8')
            conn.setdecoding(pyodbc.SQL_WCHAR, encoding='utf-8')
            conn.setencoding(encoding='utf-8')
            
            # Use pandas read_sql for better performance
            start_time = time.time()
            df = pd.read_sql(SQL_QUERY, conn)
            logger.info(f"Query executed in {time.time() - start_time:.2f} seconds")
            
            return df, None
    except Exception as e:
        error_msg = f"Error fetching data: {str(e)}"
        logger.error(error_msg, exc_info=True)
        return None, error_msg

def save_to_csv(df: pd.DataFrame, filepath: str) -> None:
    """Save DataFrame to CSV with error handling."""
    try:
        # Ensure directory exists
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        
        # Save with optimized settings
        df.to_csv(filepath, index=False, compression='infer')
        logger.info(f"Data saved to {filepath}")
    except Exception as e:
        logger.error(f"Error saving to CSV: {str(e)}", exc_info=True)
        raise

def load_data() -> pd.DataFrame:
    """Main function to load data with caching mechanism."""
    try:
        # Try loading from cache first
        if os.path.exists(CSV_FILE_PATH):
            logger.info(f"Loading data from cache: {CSV_FILE_PATH}")
            return pd.read_csv(CSV_FILE_PATH)
        
        # Fetch from database if cache doesn't exist
        logger.info("Cache not found, fetching from database")
        df, error = fetch_data_from_db()
        
        if error:
            raise RuntimeError(error)
        
        if df is not None and not df.empty:
            # Save to cache
            save_to_csv(df, CSV_FILE_PATH)
            return df
        else:
            raise ValueError("No data retrieved from database")
            
    except Exception as e:
        logger.error(f"Error in load_data: {str(e)}", exc_info=True)
        raise

# Main execution
try:
    df = load_data()
    logger.info(f"DataFrame shape: {df.shape}")
except Exception as e:
    logger.error("Failed to load data", exc_info=True)
    raise

In [ ]:
print(df.shape)
df.head()


In [ ]:


# Configure logging if not already configured
logger = logging.getLogger(__name__)

# Define columns to combine
columns_to_combine = ['wrkordr_wrk_prfrmd', 'wrkordreqpmnt_wrk_prfrmd', 'wrkordrinvntry_dscrptn']

# Pre-compile the regex pattern for better performance
pattern = re.compile(r'[^a-zA-Z0-9\s]|\r')

try:
    # Make a copy of only source columns
    df = df[columns_to_combine].copy()

    # Perform operations in a memory-efficient way
    df['combined_column'] = (
        df[columns_to_combine]
        .fillna('')  # Replace NaN with empty string
        .astype(str)  # Convert to string type (more compatible than string[pyarrow])
        .agg(' '.join, axis=1)  # Join columns with space
        .str.replace(pattern, ' ', regex=True)  # Clean text using pre-compiled pattern
    )

    # Clean up extra whitespace
    df['combined_column'] = df['combined_column'].str.strip().str.replace(r'\s+', ' ')

    # Optional: Free memory if the original columns are no longer needed
    if columns_to_combine:
        df.drop(columns=columns_to_combine, inplace=True)

except Exception as e:
    logger.error(f"Error processing DataFrame: {str(e)}")
    raise



In [ ]:
# Select a random sample of 100,000 records from the DataFrame
df_sample = df.sample(n=10000, random_state=42)
# df_sample = df

In [ ]:
from functools import lru_cache
import torch
from transformers import (
    BertTokenizer, 
    BertForTokenClassification, 
    logging as transformers_logging,
    AutoTokenizer,
    AutoModelForTokenClassification
)
import logging
from typing import Tuple, Optional
import os
import sys
from transformers import __version__ as transformers_version

# Configure detailed logging
logging.basicConfig(
    level=logging.DEBUG,  # Changed to DEBUG for more detailed logs
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)
transformers_logging.set_verbosity_info()

def check_environment():
    """Check and log environment details."""
    logger.info(f"Python version: {sys.version}")
    logger.info(f"PyTorch version: {torch.__version__}")
    logger.info(f"Transformers version: {transformers_version}")
    logger.info(f"CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        logger.info(f"CUDA version: {torch.version.cuda}")
        logger.info(f"GPU device: {torch.cuda.get_device_name(0)}")

@lru_cache(maxsize=1)
def get_device() -> torch.device:
    """Determine the optimal device with caching."""
    try:
        if torch.cuda.is_available():
            logger.info("CUDA device detected")
            return torch.device("cuda")
        elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
            logger.info("MPS device detected")
            return torch.device("mps")
        logger.info("Using CPU device")
        return torch.device("cpu")
    except Exception as e:
        logger.warning(f"Error detecting device, defaulting to CPU: {str(e)}")
        return torch.device("cpu")

def load_model_components(model_name: str, device: torch.device) -> Tuple[Optional[BertTokenizer], Optional[BertForTokenClassification]]:
    """Load model components with optimizations and proper error handling."""
    try:
        # Set up caching directory
        cache_dir = os.path.join(os.path.expanduser("~"), ".cache", "huggingface")
        os.makedirs(cache_dir, exist_ok=True)
        logger.info(f"Using cache directory: {cache_dir}")

        # Try loading with Auto classes first
        logger.info("Attempting to load tokenizer...")
        try:
            tokenizer = AutoTokenizer.from_pretrained(
                model_name,
                cache_dir=cache_dir,
                use_fast=True,
                model_max_length=512
            )
        except Exception as e:
            logger.warning(f"AutoTokenizer failed, trying BertTokenizer: {str(e)}")
            tokenizer = BertTokenizer.from_pretrained(
                model_name,
                cache_dir=cache_dir,
                use_fast=True,
                model_max_length=512
            )
        logger.info("Tokenizer loaded successfully")

        # Load model with Auto class
        logger.info("Attempting to load model...")
        try:
            model = AutoModelForTokenClassification.from_pretrained(
                model_name,
                cache_dir=cache_dir,
                return_dict=True,  # Changed to True for better compatibility
                low_cpu_mem_usage=True
            )
        except Exception as e:
            logger.warning(f"AutoModel failed, trying BertForTokenClassification: {str(e)}")
            model = BertForTokenClassification.from_pretrained(
                model_name,
                cache_dir=cache_dir,
                return_dict=True,
                low_cpu_mem_usage=True
            )
        logger.info("Model loaded successfully")

        # Move model to device and optimize
        logger.info(f"Moving model to device: {device}")
        model = model.to(device)
        model.eval()
        
        # Disable gradient computation for inference
        torch.set_grad_enabled(False)
        
        return tokenizer, model

    except Exception as e:
        logger.error(f"Error loading model components: {str(e)}", exc_info=True)
        return None, None

def initialize_model() -> Tuple[Optional[BertTokenizer], Optional[BertForTokenClassification], Optional[torch.device]]:
    """Initialize model with proper error handling and optimizations."""
    try:
        # Check environment first
        check_environment()
        
        # Get device (cached)
        device = get_device()
        logger.info(f"Using device: {device}")

        # Try alternative model if the original fails
        model_names = [
            'vblagoje/bert-english-uncased-finetuned-pos',
            'bert-base-uncased'  # Fallback model
        ]
        
        for model_name in model_names:
            logger.info(f"Attempting to load model: {model_name}")
            tokenizer, model = load_model_components(model_name, device)
            
            if tokenizer is not None and model is not None:
                logger.info(f"Successfully loaded model: {model_name}")
                return tokenizer, model, device
            
            logger.warning(f"Failed to load model: {model_name}, trying next option...")
        
        raise RuntimeError("All model loading attempts failed")

    except Exception as e:
        logger.error(f"Error initializing model: {str(e)}", exc_info=True)
        return None, None, None

# Initialize the model with proper error handling
try:
    # Clear any existing cache
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    tokenizer, model, device = initialize_model()
    if None in (tokenizer, model, device):
        raise RuntimeError("Model initialization failed")
    logger.info("Model initialized successfully")
except Exception as e:
    logger.error("Failed to initialize model", exc_info=True)
    raise

In [ ]:
from functools import lru_cache
import re
from typing import List, Tuple, Optional
from dataclasses import dataclass

@dataclass
class ProcessingContext:
    current_verb: Optional[str] = None
    current_objects: List[str] = None
    
    def __post_init__(self):
        if self.current_objects is None:
            self.current_objects = []

    def reset(self):
        self.current_verb = None
        self.current_objects.clear()

@lru_cache(maxsize=1024)
def is_potential_verb(word: str, tag: str) -> bool:
    """Cached check for potential verbs."""
    return (tag.startswith('VB') or tag == 'VBD' or tag == 'VBN' or 
            (tag == 'JJ' and (word.endswith('ed') or word.endswith('en'))))

@lru_cache(maxsize=1024)
def is_object_component(tag: str, word: str) -> bool:
    """Cached check for object components."""
    return (tag.startswith('NN') or 
            (tag.startswith('JJ') and not word.endswith('ed')) or 
            tag == 'IN' or tag.startswith('VBG'))

def preprocess_text(text: str) -> str:
    """Preprocess text with compiled regex patterns."""
    # Compile regex patterns once
    PATTERNS = {
        'as_needed': re.compile(r'\bas needed\b'),
        'conjunctions': re.compile(r'\s*,\s*and\s+'),
        'whitespace': re.compile(r'\s+')
    }
    
    text = text.lower()
    text = PATTERNS['as_needed'].sub('', text)
    text = PATTERNS['conjunctions'].sub(', ', text)
    return PATTERNS['whitespace'].sub(' ', text).strip()

def process_compound_objects(tagged: List[Tuple[str, str]], start_idx: int) -> Tuple[List[str], int]:
    """Process compound objects and return objects and new index."""
    objects = [tagged[start_idx][0]]
    i = start_idx + 1
    
    while i < len(tagged) and is_object_component(tagged[i][1], tagged[i][0]):
        objects.append(tagged[i][0])
        i += 1
    
    return objects, i - 1

def process_text(text: str, tokenizer, model, device) -> List[Tuple[str, str]]:
    try:
        # Initialize tools lazily (only when needed)
        spell = SpellChecker()
        lemmatizer = WordNetLemmatizer()
        
        # Preprocess text
        text = preprocess_text(text)
        
        # Process text
        tokens = word_tokenize(text)
        tagged = pos_tag(tokens)
        
        verb_object_pairs = []
        context = ProcessingContext()
        
        def add_pair():
            if context.current_verb and context.current_objects:
                lemmatized_verb = lemmatizer.lemmatize(context.current_verb, 'v')
                verb_object_pairs.append((lemmatized_verb, ' '.join(context.current_objects)))
        
        i = 0
        while i < len(tagged):
            word, tag = tagged[i]
            
            if is_potential_verb(word, tag):
                # Save previous pair if exists
                add_pair()
                context.current_verb = spell.correction(word)
                context.current_objects.clear()
                
            elif is_object_component(tag, word):
                if context.current_verb:
                    temp_objects, new_idx = process_compound_objects(tagged, i)
                    if any(tagged[j][1].startswith('NN') for j in range(i, new_idx + 1)):
                        context.current_objects.extend(temp_objects)
                    i = new_idx
                    
            elif word == ',':
                add_pair()
                context.reset()
            
            i += 1
        
        # Add final pair if exists
        add_pair()
        
        return verb_object_pairs

    except Exception as e:
        import traceback
        print(f"Error processing text: {str(e)}")
        print(f"Traceback: {traceback.format_exc()}")
        return []

In [ ]:
def categorize_task(verb,obj):
    """Categorize a task based on its verb into one of four categories: Inspection, Cleaning, Maintenance, or Other."""
    inspection_verbs = {'inspect', 'check', 'examine', 'monitor', 'observe', 'verify', 'test', 'diagnose', 'assess'}
    cleaning_verbs = {'clean', 'brush', 'wash', 'wipe', 'descale', 'sanitize', 'flush', 'clear'}
    maintenance_verbs = {'lubricate', 'adjust', 'replace', 'repair', 'calibrate', 'install', 'fix', 'tighten', 'service'}
    
    verb = verb.lower()
    if verb in inspection_verbs:
        return "Inspection"
    elif verb in cleaning_verbs:
        return "Cleaning"
    elif verb in maintenance_verbs:
        return "Maintenance"
    else:
        return "Other"

In [ ]:
def analyze_root_cause(verb, obj, text):
    """Analyze the root cause of an issue based on the maintenance action and context."""
    # Define component categories and their related terms
    components = {
        'ice_making': {'ice', 'cube', 'evaporator', 'freezing', 'harvest'},
        'refrigeration': {'refrigerant', 'compressor', 'condenser', 'cooling', 'freon', 'leak'},
        'water_system': {'water', 'filter', 'pump', 'valve', 'flow', 'drain'},
        'electrical': {'power', 'circuit', 'electrical', 'switch', 'sensor', 'board'},
        'mechanical': {'belt', 'motor', 'bearing', 'gear', 'pulley', 'fan'}
    }
    
    # Define problem indicators
    problem_indicators = {
        'failure': {'failed', 'broken', 'not working', 'stopped', 'dead'},
        'performance': {'slow', 'weak', 'inefficient', 'poor', 'reduced'},
        'noise': {'loud', 'noisy', 'vibration', 'rattling', 'squealing'},
        'leakage': {'leak', 'dripping', 'overflow', 'spill'},
        'quality': {'dirty', 'contaminated', 'scale', 'buildup', 'poor quality'}
    }
    
    def find_matches(text, term_dict):
        text_lower = text.lower()
        matches = []
        for category, terms in term_dict.items():
            if any(term in text_lower for term in terms):
                matches.append(category)
        return matches
    
    # Analyze components involved
    affected_components = find_matches(text, components)
    
    # Analyze problem types
    problem_types = find_matches(text, problem_indicators)
    
    # Determine maintenance type
    if verb.lower() in {'replace', 'install', 'rebuild'}:
        maintenance_type = 'replacement'
    elif verb.lower() in {'clean', 'flush', 'sanitize'}:
        maintenance_type = 'cleaning'
    elif verb.lower() in {'adjust', 'calibrate', 'tune'}:
        maintenance_type = 'adjustment'
    elif verb.lower() in {'repair', 'fix', 'patch'}:
        maintenance_type = 'repair'
    else:
        maintenance_type = 'other'
    
    return {
        'components': affected_components,
        'problem_types': problem_types,
        'maintenance_type': maintenance_type,
        'action': f"{verb} {obj}"
    }

def get_component_statistics(maintenance_records):
    """Generate statistics about component issues and maintenance patterns."""
    component_stats = {
        'component_failures': Counter(),
        'problem_types': Counter(),
        'maintenance_types': Counter(),
        'recurring_issues': defaultdict(list)
    }
    
    for record in maintenance_records:
        analysis = analyze_root_cause(record['verb'], record['object'], record['text'])
        
        # Count component failures
        for component in analysis['components']:
            component_stats['component_failures'][component] += 1
        
        # Count problem types
        for problem in analysis['problem_types']:
            component_stats['problem_types'][problem] += 1
            
        # Count maintenance types
        component_stats['maintenance_types'][analysis['maintenance_type']] += 1
        
        # Track recurring issues
        if analysis['components'] and analysis['problem_types']:
            issue_key = f"{','.join(analysis['components'])}:{','.join(analysis['problem_types'])}"
            component_stats['recurring_issues'][issue_key].append(record['date'])
    
    return component_stats

In [ ]:
# Process all records in df_sample['combined_column']
from collections import Counter
from collections import defaultdict
from tqdm.notebook import tqdm
import time
from datetime import datetime, timedelta

print("Starting to process maintenance records...")
start_time = time.time()

# Initialize counters and storage
all_verb_object_pairs = []  # Keep as list to track frequencies
task_frequencies = Counter()
maintenance_records = []

# Process each record
records = df_sample['combined_column'].dropna()
total_records = len(records)
print(f"Total records to process: {total_records:,}")

# Initialize progress bar with additional metrics
pbar = tqdm(records.items(), 
            total=total_records,
            desc="Processing records",
            unit="records",
            bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]")

processed_pairs = 0
for date_idx, text in pbar:
    pairs = process_text(text, tokenizer, model, device)
    if pairs:  # Only add if we got valid pairs
        for pair in pairs:
            verb, obj = pair
            task_frequencies[tuple(pair)] += 1
            all_verb_object_pairs.append(tuple(pair))
            processed_pairs += 1
            
            # Add record for root cause analysis
            maintenance_records.append({
                'verb': verb,
                'object': obj,
                'text': text,
                'date': date_idx  # Now using the actual index from items()
            })
    
    # Update progress bar description with additional metrics
    if processed_pairs % 100 == 0:  # Update every 100 records to avoid overhead
        elapsed = time.time() - start_time
        rate = processed_pairs / elapsed if elapsed > 0 else 0
        remaining_pairs = total_records - processed_pairs
        eta = remaining_pairs / rate if rate > 0 else 0
        eta_str = str(timedelta(seconds=int(eta)))
        
        pbar.set_postfix({
            'Pairs Found': processed_pairs,
            'Avg Pairs/Record': f"{processed_pairs/len(records):.2f}",
            'ETA': eta_str
        })

end_time = time.time()
processing_time = end_time - start_time

print(f"\nProcessing completed in {timedelta(seconds=int(processing_time))}")
print(f"Total task occurrences: {len(all_verb_object_pairs):,}")
print(f"Unique tasks: {len(task_frequencies):,}")
print(f"Average processing speed: {total_records/processing_time:.1f} records/second")
print(f"Average pairs per record: {len(all_verb_object_pairs)/total_records:.2f}")

# Group tasks by category with frequencies
tasks_by_category = {}
category_totals = {}

for (verb, obj), freq in task_frequencies.most_common():
    category = categorize_task(verb, obj)
    if category not in tasks_by_category:
        tasks_by_category[category] = []
        category_totals[category] = 0
    tasks_by_category[category].append((verb, obj, freq))
    category_totals[category] += freq

# Calculate total tasks
total_tasks = sum(category_totals.values())

# Create summary DataFrame
summary_data = {
    'Category': [],
    'Total_Tasks': [],
    'Unique_Tasks': [],
    'Percentage': []
}

for category in sorted(category_totals.keys()):
    summary_data['Category'].append(category)
    summary_data['Total_Tasks'].append(category_totals[category])
    summary_data['Unique_Tasks'].append(len(tasks_by_category[category]))
    summary_data['Percentage'].append((category_totals[category] / total_tasks) * 100)

summary_df = pd.DataFrame(summary_data)
print("\nSummary DataFrame:")
print(summary_df)

# After processing all records, analyze root causes
print("\nAnalyzing root causes...")
component_stats = get_component_statistics(maintenance_records)

# Print root cause analysis results
print("\nComponent Failure Analysis:")
for component, count in component_stats['component_failures'].most_common():
    print(f"{component}: {count:,} occurrences")

print("\nProblem Type Analysis:")
for problem, count in component_stats['problem_types'].most_common():
    print(f"{problem}: {count:,} occurrences")

print("\nMaintenance Type Distribution:")
for mtype, count in component_stats['maintenance_types'].most_common():
    print(f"{mtype}: {count:,} occurrences")

print("\nTop Recurring Issues:")
recurring_sorted = sorted(
    component_stats['recurring_issues'].items(), 
    key=lambda x: len(x[1]), 
    reverse=True
)
for issue, dates in recurring_sorted[:10]:
    components, problems = issue.split(':')
    print(f"Components: {components}")
    print(f"Problems: {problems}")
    print(f"Occurrences: {len(dates):,}\n")

In [ ]:
import os
import pandas as pd

# Create data directory if it doesn't exist
os.makedirs('data/maintenance_analysis', exist_ok=True)

# Set pandas display options to prevent wrapping and ensure consistent formatting
pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: f'{x:,}')

# Create DataFrames for each category
category_dfs = {}
for category in ['Maintenance', 'Inspection', 'Cleaning', 'Other']:
    if category in tasks_by_category:
        category_tasks = []
        for verb, obj, freq in tasks_by_category[category]:
            # Get root cause info
            maintenance_record = next((r for r in maintenance_records if r['verb'] == verb and r['object'] == obj), None)
            if maintenance_record:
                analysis = analyze_root_cause(verb, obj, maintenance_record['text'])
                category_tasks.append({
                    'Verb': verb,
                    'Object': obj,
                    'Frequency': freq,
                    'Task Description': f"{verb} {obj}".strip(),
                    'Components': ', '.join(analysis['components']) if analysis['components'] else 'Other',
                    'Problem Types': ', '.join(analysis['problem_types']) if analysis['problem_types'] else 'Unknown',
                    'Maintenance Type': analysis['maintenance_type']
                })
    
        if category_tasks:
            category_dfs[category] = pd.DataFrame(category_tasks)
            category_dfs[category] = category_dfs[category].sort_values('Frequency', ascending=False)
            
            print(f"\n{category} Tasks:")
            print("=" * 50)
            
            # Format the DataFrame display
            df_display = category_dfs[category].head(10).copy()
            df_display.index = range(1, len(df_display) + 1)
            print(df_display.to_string(
                justify='left',
                col_space={
                    'Verb': 15,
                    'Object': 30,
                    'Frequency': 12,
                    'Task Description': 35,
                    'Components': 20,
                    'Problem Types': 25,
                    'Maintenance Type': 15
                }
            ))

# Save analysis results
save_dir = 'data/maintenance_analysis'

# Save category DataFrames
for category, df in category_dfs.items():
    if not df.empty:
        filename = os.path.join(save_dir, f"{category.lower()}_tasks.csv")
        df.to_csv(filename, index=False)
        print(f"\nSaved {category} tasks to {filename}")

# Save component analysis
component_df = pd.DataFrame({
    'Component': list(component_stats['component_failures'].keys()),
    'Failures': list(component_stats['component_failures'].values())
}).sort_values('Failures', ascending=False)

filename = os.path.join(save_dir, 'component_analysis.csv')
component_df.to_csv(filename, index=False)
print(f"\nSaved component analysis to {filename}")

# Save problem type analysis
problem_df = pd.DataFrame({
    'Problem Type': list(component_stats['problem_types'].keys()),
    'Occurrences': list(component_stats['problem_types'].values())
}).sort_values('Occurrences', ascending=False)

filename = os.path.join(save_dir, 'problem_analysis.csv')
problem_df.to_csv(filename, index=False)
print(f"\nSaved problem type analysis to {filename}")

In [ ]:
# Display summary of all tasks
print(f"\nMaintenance Tasks Analysis Summary ({pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')})")
print("=" * 120)

# Create a combined DataFrame of all tasks
all_tasks = []
for category, df in category_dfs.items():
    df = df.copy()
    df['Category'] = category
    all_tasks.append(df)
all_tasks_df = pd.concat(all_tasks, ignore_index=True)

# Create task summary DataFrame with formatting
summary_df = all_tasks_df.groupby('Category').agg({
    'Task Description': 'count',
    'Frequency': 'sum'
}).rename(columns={
    'Task Description': 'Unique Tasks',
    'Frequency': 'Total Occurrences'
})

# Add percentage columns for tasks
total_tasks = summary_df['Unique Tasks'].sum()
total_occurrences = summary_df['Total Occurrences'].sum()
summary_df['% of Tasks'] = (summary_df['Unique Tasks'] / total_tasks * 100).round(1)
summary_df['% of Occurrences'] = (summary_df['Total Occurrences'] / total_occurrences * 100).round(1)

# Component analysis summary
component_summary = pd.DataFrame.from_dict(
    component_stats['component_failures'], 
    orient='index',
    columns=['Failures']
).sort_values('Failures', ascending=False)
component_summary['% of Failures'] = (component_summary['Failures'] / component_summary['Failures'].sum() * 100).round(1)

# Problem type analysis summary
problem_summary = pd.DataFrame.from_dict(
    component_stats['problem_types'], 
    orient='index',
    columns=['Occurrences']
).sort_values('Occurrences', ascending=False)
problem_summary['% of Problems'] = (problem_summary['Occurrences'] / problem_summary['Occurrences'].sum() * 100).round(1)

# Maintenance type distribution
maintenance_types = all_tasks_df['Maintenance Type'].value_counts()
maintenance_summary = pd.DataFrame({
    'Count': maintenance_types,
    '% of Total': (maintenance_types / len(all_tasks_df) * 100).round(1)
})

# Format numbers with thousands separator
summary_df['Unique Tasks'] = summary_df['Unique Tasks'].apply(lambda x: f"{x:,}")
summary_df['Total Occurrences'] = summary_df['Total Occurrences'].apply(lambda x: f"{x:,}")

# Sort categories in logical order
category_order = ['Maintenance', 'Inspection', 'Cleaning', 'Other']
summary_df = summary_df.reindex(category_order)

# Set display options for better formatting
pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: f'{x:,.0f}' if x.is_integer() else f'{x:.1f}%')

# Print all summaries
print("\nTask Category Summary:")
print("-" * 80)
print(summary_df)
print("\nComponent Failure Summary (Top 5):")
print("-" * 80)
print(component_summary.head())
print("\nProblem Type Summary (Top 5):")
print("-" * 80)
print(problem_summary.head())
print("\nMaintenance Type Distribution:")
print("-" * 80)
print(maintenance_summary)
print("=" * 80)
print(f"Total Unique Tasks: {total_tasks:,.0f}")
print(f"Total Task Occurrences: {total_occurrences:,.0f}")
print(f"Total Components Analyzed: {len(component_summary):,.0f}")
print(f"Total Problem Types Identified: {len(problem_summary):,.0f}")
print("=" * 80)

In [ ]:
# Create DataFrames for root cause analysis
print("Detailed Root Cause Analysis")
print("=" * 100)

# 1. Component Failures by Subsystem
component_df = pd.DataFrame.from_dict(
    component_stats['component_failures'], 
    orient='index',
    columns=['Count']
).sort_values('Count', ascending=False)

# Group components by subsystem
subsystem_mapping = {
    'freeze_plate_system': 'Ice Making',
    'ice_formation_control': 'Ice Making',
    'harvest_system': 'Ice Making',
    'ice_handling': 'Ice Making',
    'water_distribution_ice': 'Ice Making',
    'compressor': 'Refrigeration',
    'condenser': 'Refrigeration',
    'refrigerant_circuit': 'Refrigeration',
    'expansion_device': 'Refrigeration',
    'water_supply': 'Water System',
    'water_pump': 'Water System',
    'water_distribution': 'Water System',
    'drain_system': 'Water System',
    'control_board': 'Electrical',
    'sensors': 'Electrical',
    'electrical_components': 'Electrical',
    'wiring': 'Electrical',
    'motors': 'Mechanical',
    'belts_chains': 'Mechanical',
    'framework': 'Mechanical'
}

component_df['Subsystem'] = component_df.index.map(lambda x: subsystem_mapping.get(x, 'Other'))
component_df['Percentage'] = (component_df['Count'] / component_df['Count'].sum() * 100).round(1)

# Group by subsystem for summary
subsystem_summary = component_df.groupby('Subsystem').agg({
    'Count': 'sum',
    'Percentage': 'sum'
}).sort_values('Count', ascending=False)

print("\nFailures by Subsystem:")
print("-" * 100)
print(subsystem_summary.to_string(float_format=lambda x: f'{x:,.1f}'))

print("\nTop 10 Specific Component Failures:")
print("-" * 100)
print(component_df.head(10).to_string(float_format=lambda x: f'{x:,.1f}'))

# 2. Problem Types with Severity
problem_df = pd.DataFrame.from_dict(
    component_stats['problem_types'], 
    orient='index',
    columns=['Count']
).sort_values('Count', ascending=False)

problem_df['Percentage'] = (problem_df['Count'] / problem_df['Count'].sum() * 100).round(1)

print("\nProblem Types by Frequency:")
print("-" * 100)
print(problem_df.to_string(float_format=lambda x: f'{x:,.1f}'))

# 3. Recurring Issues Analysis
recurring_issues = []
for issue, dates in component_stats['recurring_issues'].items():
    components, problems = issue.split(':')
    
    # Convert dates to datetime if they aren't already
    date_objects = []
    for date in dates:
        if isinstance(date, (int, float)):
            # If date is a timestamp
            date_obj = pd.to_datetime(date, unit='s')
        elif isinstance(date, str):
            # If date is a string
            date_obj = pd.to_datetime(date)
        else:
            # If date is already datetime
            date_obj = date
        date_objects.append(date_obj)
    
    if date_objects:
        mtbf_days = (max(date_objects) - min(date_objects)).total_seconds() / (86400 * len(date_objects))
        last_occurrence = max(date_objects).strftime('%Y-%m-%d')
    else:
        mtbf_days = 0
        last_occurrence = 'N/A'
    
    recurring_issues.append({
        'Components': components,
        'Problems': problems,
        'Occurrences': len(dates),
        'Mean Time Between Failures': f"{mtbf_days:.1f} days",
        'Last Occurrence': last_occurrence
    })

recurring_df = pd.DataFrame(recurring_issues)
if not recurring_df.empty:
    recurring_df = recurring_df.sort_values('Occurrences', ascending=False)
    
    print("\nTop Recurring Issues (with MTBF):")
    print("-" * 100)
    print(recurring_df.head(10).to_string(index=False))

# Save detailed analysis to CSV
output_dir = 'data/maintenance_analysis'
os.makedirs(output_dir, exist_ok=True)

component_df.to_csv(f'{output_dir}/component_failures_detailed.csv')
subsystem_summary.to_csv(f'{output_dir}/subsystem_summary.csv')
problem_df.to_csv(f'{output_dir}/problem_types_detailed.csv')
if not recurring_df.empty:
    recurring_df.to_csv(f'{output_dir}/recurring_issues_detailed.csv', index=False)

print(f"\nDetailed analysis files saved to: {output_dir}")

In [ ]:
def analyze_root_cause(verb, obj, text):
    """Analyze the root cause of an issue based on the maintenance action and context."""
    # Define detailed component categories and their related terms
    components = {
        # Ice Making System
        'freeze_plate_system': {
            'evaporator plate', 'freeze plate', 'cooling surface', 'grid plate', 
            'ice mold', 'cube form', 'ice formation', 'freezing surface',
            'water distribution plate', 'cell size', 'bridge thickness'
        },
        'ice_formation_control': {
            'thickness sensor', 'ice thickness', 'bridge control',
            'water level sensor', 'formation time', 'freeze cycle',
            'water curtain', 'spray time', 'freeze timer'
        },
        'harvest_system': {
            'harvest valve', 'hot gas valve', 'defrost valve', 
            'harvest timer', 'harvest assist', 'release mechanism',
            'hot gas bypass', 'harvest pressure', 'harvest solenoid',
            'harvest check valve', 'harvest complete switch'
        },
        'ice_handling': {
            'ice bin', 'storage bin', 'bin thermostat', 'bin control',
            'ice chute', 'dispenser', 'agitator', 'auger motor',
            'ice gate', 'bin door', 'bin level sensor', 'ice shield'
        },
        'water_distribution_ice': {
            'water pump', 'spray bar', 'spray nozzles', 'water curtain',
            'water trough', 'water sump', 'distribution tube',
            'water level', 'float valve', 'reservoir'
        },
        
        # Refrigeration System
        'compressor_system': {
            'compressor', 'piston', 'scroll', 'reed valve', 'valve plate',
            'discharge line', 'suction line', 'oil level', 'crankcase',
            'compressor motor', 'windings', 'terminal', 'overload protector'
        },
        'condenser_system': {
            'condenser coil', 'fan motor', 'fan blade', 'air flow',
            'head pressure', 'subcooling', 'fins', 'coil surface',
            'condenser pressure', 'air cooled', 'water cooled'
        },
        'refrigerant_circuit': {
            'refrigerant', 'freon', 'charge', 'leak', 'pressure',
            'filter drier', 'sight glass', 'accumulator', 'receiver',
            'liquid line', 'suction line', 'discharge line'
        },
        'expansion_system': {
            'TXV', 'expansion valve', 'capillary tube', 'orifice',
            'superheat', 'bulb', 'equalizer', 'distributor',
            'metering device', 'restrictor'
        },
        
        # Water System
        'water_supply': {
            'water line', 'inlet valve', 'water pressure', 'filter',
            'strainer', 'softener', 'water quality', 'supply pipe',
            'shut off valve', 'pressure regulator'
        },
        'water_pump_system': {
            'circulation pump', 'impeller', 'seal', 'pump motor',
            'pump capacity', 'pump housing', 'bearing', 'shaft',
            'pump pressure', 'pump strainer'
        },
        'water_distribution': {
            'spray nozzles', 'water curtain', 'distributor', 'tube',
            'spray pattern', 'distribution manifold', 'spray bar',
            'water flow', 'distribution uniformity'
        },
        'drain_system': {
            'drain line', 'condensate', 'drain pan', 'float switch',
            'pump out', 'drain valve', 'trap', 'vent', 'slope',
            'drain heater', 'overflow protection'
        },
        
        # Electrical System
        'control_board': {
            'PCB', 'control board', 'controller', 'motherboard',
            'processor', 'relay board', 'display board', 'interface',
            'memory', 'firmware', 'programming'
        },
        'sensors_system': {
            'thermistor', 'probe', 'sensor', 'thermostat', 'float switch',
            'bin level sensor', 'pressure sensor', 'temperature sensor',
            'water level sensor', 'ice thickness sensor'
        },
        'electrical_components': {
            'contactor', 'relay', 'capacitor', 'transformer', 'fuse',
            'overload', 'terminal block', 'wire connector', 'breaker',
            'power supply', 'voltage regulator'
        },
        'wiring_system': {
            'wire harness', 'connection', 'terminal', 'plug', 'socket',
            'ground wire', 'power cable', 'communication wire',
            'insulation', 'conduit', 'junction box'
        }
    }
    
    # Define specific problem types with detailed indicators
    problem_indicators = {
        'mechanical_failure': {
            'seized', 'stuck', 'broken', 'cracked', 'worn', 'misaligned',
            'loose', 'bent', 'damaged', 'jammed', 'binding', 'stripped'
        },
        'electrical_failure': {
            'short', 'open circuit', 'voltage', 'tripped', 'burnt',
            'no power', 'grounded', 'overload', 'electrical noise',
            'intermittent power', 'voltage drop'
        },
        'ice_quality_issue': {
            'cloudy ice', 'small cubes', 'incomplete cubes', 'malformed',
            'hollow cubes', 'white spots', 'bridging', 'long harvest',
            'slow production', 'uneven size'
        },
        'performance_issue': {
            'slow', 'inefficient', 'reduced', 'low production',
            'poor quality', 'inconsistent', 'erratic', 'unstable',
            'degraded', 'below spec'
        },
        'leakage_issue': {
            'leak', 'drip', 'overflow', 'flood', 'seal failure',
            'gasket leak', 'water loss', 'refrigerant leak',
            'oil leak', 'seepage'
        },
        'contamination': {
            'dirty', 'scale', 'calcium', 'mineral', 'debris', 'buildup',
            'corrosion', 'rust', 'slime', 'mold', 'algae', 'sediment'
        },
        'noise_vibration': {
            'noise', 'vibration', 'rattle', 'squeal', 'bang', 'hum',
            'grinding', 'knocking', 'whistling', 'clicking', 'rumbling'
        },
        'control_system': {
            'error code', 'not responding', 'incorrect', 'erratic',
            'intermittent', 'display error', 'sensor error', 'communication error',
            'program error', 'calibration error'
        },
        'temperature_issue': {
            'hot', 'cold', 'warm', 'freezing', 'not cooling',
            'high temp', 'low temp', 'inconsistent temp',
            'temperature swing', 'poor temperature control'
        },
        'pressure_issue': {
            'high pressure', 'low pressure', 'no pressure', 'pressure drop',
            'pressure fluctuation', 'excessive pressure', 'insufficient pressure',
            'pressure lockout', 'pressure control'
        }
    }
    
    # Define maintenance types with detailed activities
    maintenance_types = {
        'preventive': {
            'inspect', 'clean', 'adjust', 'lubricate', 'tighten',
            'calibrate', 'test', 'check', 'measure', 'scheduled',
            'routine', 'preventative'
        },
        'corrective': {
            'repair', 'replace', 'fix', 'rebuild', 'overhaul',
            'restore', 'rectify', 'correct', 'resolve', 'service'
        },
        'predictive': {
            'monitor', 'analyze', 'trend', 'forecast', 'predict',
            'assess', 'evaluate', 'diagnose', 'investigate', 'study'
        },
        'emergency': {
            'breakdown', 'failure', 'emergency', 'urgent', 'critical',
            'immediate', 'unplanned', 'unexpected', 'sudden', 'acute'
        }
    }
    
    def find_matches(text, term_dict):
        """Find matches in text for terms in term_dict."""
        text_lower = text.lower()
        matches = []
        for category, terms in term_dict.items():
            if any(term.lower() in text_lower for term in terms):
                matches.append(category)
        return matches
    
    # Combine all text fields for analysis
    full_text = f"{text} {obj} {verb}".lower()
    
    # Find all matches
    found_components = find_matches(full_text, components)
    found_problems = find_matches(full_text, problem_indicators)
    
    # Determine maintenance type
    maint_matches = find_matches(full_text, maintenance_types)
    maintenance_type = maint_matches[0] if maint_matches else 'unspecified'
    
    # Determine severity based on various factors
    severity_indicators = {
        'high': {'emergency', 'critical', 'urgent', 'failure', 'breakdown', 'safety',
                'immediate', 'severe', 'major', 'serious'},
        'medium': {'degraded', 'warning', 'attention', 'minor failure', 'reduced',
                  'inconsistent', 'poor', 'issue'},
        'low': {'routine', 'normal', 'scheduled', 'preventive', 'adjust', 'check',
                'inspect', 'clean', 'monitor'}
    }
    
    # Determine severity
    severity = 'normal'
    for level, indicators in severity_indicators.items():
        if any(indicator in full_text for indicator in indicators):
            severity = level
            break
    
    return {
        'components': found_components,
        'problem_types': found_problems,
        'maintenance_type': maintenance_type,
        'severity': severity,
        'timestamp': pd.Timestamp.now()  # Add timestamp for tracking
    }

In [ ]:
# Required imports
from collections import defaultdict
from tqdm import tqdm
import pandas as pd
import os

# Perform root cause analysis on maintenance records
print("Detailed Root Cause Analysis")
print("=" * 100)

# Initialize counters for analysis
component_failures = defaultdict(int)
problem_types = defaultdict(int)
maintenance_types = defaultdict(int)
severity_counts = defaultdict(int)
recurring_issues = defaultdict(list)

# Process all records
print(f"\nAnalyzing {len(maintenance_records)} maintenance records...")

for record in tqdm(maintenance_records, desc="Processing records"):
    # Perform root cause analysis
    analysis = analyze_root_cause(record['verb'], record['object'], record['text'])
    
    # Count component failures
    for component in analysis['components']:
        component_failures[component] += 1
    
    # Count problem types
    for problem in analysis['problem_types']:
        problem_types[problem] += 1
    
    # Count maintenance types
    maintenance_types[analysis['maintenance_type']] += 1
    
    # Count severity levels
    severity_counts[analysis['severity']] += 1
    
    # Track recurring issues with proper date handling
    if analysis['components'] and analysis['problem_types']:
        components_str = ','.join(sorted(list(analysis['components'])))
        problems_str = ','.join(sorted(list(analysis['problem_types'])))
        issue_key = f"{components_str}:{problems_str}"
        # Convert date to pandas Timestamp
        date = pd.to_datetime(record['date']) if not isinstance(record['date'], pd.Timestamp) else record['date']
        recurring_issues[issue_key].append(date)

# Create DataFrames for analysis
print("\n1. Component Failures Analysis")
print("-" * 80)
component_df = pd.DataFrame.from_dict(
    component_failures, 
    orient='index',
    columns=['Count']
).sort_values('Count', ascending=False)

component_df['Percentage'] = (component_df['Count'] / component_df['Count'].sum() * 100).round(1)

# Group by subsystem
subsystem_mapping = {
    'freeze_plate_system': 'Ice Making',
    'ice_formation_control': 'Ice Making',
    'harvest_system': 'Ice Making',
    'ice_handling': 'Ice Making',
    'water_distribution_ice': 'Ice Making',
    'compressor_system': 'Refrigeration',
    'condenser_system': 'Refrigeration',
    'refrigerant_circuit': 'Refrigeration',
    'expansion_system': 'Refrigeration',
    'water_supply': 'Water System',
    'water_pump_system': 'Water System',
    'water_distribution': 'Water System',
    'drain_system': 'Water System',
    'control_board': 'Electrical',
    'sensors_system': 'Electrical',
    'electrical_components': 'Electrical',
    'wiring_system': 'Electrical'
}

component_df['Subsystem'] = component_df.index.map(lambda x: subsystem_mapping.get(x, 'Other'))
subsystem_summary = component_df.groupby('Subsystem').agg({
    'Count': 'sum',
    'Percentage': 'sum'
}).sort_values('Count', ascending=False)

print("\nFailures by Subsystem:")
print(subsystem_summary.to_string(float_format=lambda x: f'{x:,.1f}'))

print("\nTop 10 Component Failures:")
print(component_df.head(10).to_string(float_format=lambda x: f'{x:,.1f}'))

# Problem Types Analysis
print("\n2. Problem Types Analysis")
print("-" * 80)
problem_df = pd.DataFrame.from_dict(
    problem_types,
    orient='index',
    columns=['Count']
).sort_values('Count', ascending=False)

problem_df['Percentage'] = (problem_df['Count'] / problem_df['Count'].sum() * 100).round(1)
print("\nProblem Type Distribution:")
print(problem_df.to_string(float_format=lambda x: f'{x:,.1f}'))

# Maintenance Types Analysis
print("\n3. Maintenance Types Analysis")
print("-" * 80)
maintenance_df = pd.DataFrame.from_dict(
    maintenance_types,
    orient='index',
    columns=['Count']
).sort_values('Count', ascending=False)

maintenance_df['Percentage'] = (maintenance_df['Count'] / maintenance_df['Count'].sum() * 100).round(1)
print("\nMaintenance Type Distribution:")
print(maintenance_df.to_string(float_format=lambda x: f'{x:,.1f}'))

# Severity Analysis
print("\n4. Severity Analysis")
print("-" * 80)
severity_df = pd.DataFrame.from_dict(
    severity_counts,
    orient='index',
    columns=['Count']
).sort_values('Count', ascending=False)

severity_df['Percentage'] = (severity_df['Count'] / severity_df['Count'].sum() * 100).round(1)
print("\nSeverity Distribution:")
print(severity_df.to_string(float_format=lambda x: f'{x:,.1f}'))

# Recurring Issues Analysis
print("\n5. Recurring Issues Analysis")
print("-" * 80)
recurring_data = []
for issue, dates in recurring_issues.items():
    if len(dates) > 1:  # Only include if it occurred multiple times
        components, problems = issue.split(':')
        # Calculate time difference
        date_range = pd.date_range(min(dates), max(dates))
        mtbf_days = (max(dates) - min(dates)).total_seconds() / (86400 * (len(dates) - 1))
        
        recurring_data.append({
            'Components': components,
            'Problems': problems,
            'Occurrences': len(dates),
            'Mean Time Between Failures': f"{mtbf_days:.1f} days",
            'First Occurrence': min(dates).strftime('%Y-%m-%d'),
            'Last Occurrence': max(dates).strftime('%Y-%m-%d')
        })

recurring_df = pd.DataFrame(recurring_data)
if not recurring_df.empty:
    recurring_df = recurring_df.sort_values('Occurrences', ascending=False)
    print("\nTop 10 Recurring Issues:")
    print(recurring_df.head(10).to_string(index=False))

# Save analysis results
output_dir = 'data/maintenance_analysis'
os.makedirs(output_dir, exist_ok=True)

component_df.to_csv(f'{output_dir}/component_failures.csv')
subsystem_summary.to_csv(f'{output_dir}/subsystem_summary.csv')
problem_df.to_csv(f'{output_dir}/problem_types.csv')
maintenance_df.to_csv(f'{output_dir}/maintenance_types.csv')
severity_df.to_csv(f'{output_dir}/severity_analysis.csv')
recurring_df.to_csv(f'{output_dir}/recurring_issues.csv', index=False)

print(f"\nAnalysis files saved to: {output_dir}")